In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import OneHotEncoder

import pickle

In [2]:
throws = pd.read_csv("../data/throws_clean.csv")


# print(throws.head())
throw_types = pd.get_dummies(throws.throw_type).rename(columns = {"BH":"backhand","F":"forehand","H":"hammer"})

throw_analysis = pd.concat([throws,throw_types], axis = 1)[["Xdist","Ydist","backhand","forehand","hammer","completion"]].dropna()


# throws = pd.concat(throws)

# print(throw_types)


throw_analysis.head()

,Xdist,Ydist,backhand,forehand,hammer,completion
0,8.0,0.0,1,0,0,1.0
1,-1.0,16.0,1,0,0,1.0
2,7.0,9.0,1,0,0,1.0
3,14.0,4.0,1,0,0,1.0
4,-12.0,-4.0,0,1,0,1.0


In [3]:
def split_data(data, train_share = 0.8, response_col = "completion", random_state = 0):
    feature_cols = data.drop(columns = response_col)
    response = data[[response_col]]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(feature_cols, response,
                                                                                train_size=train_share,
                                                                                random_state=random_state)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(throw_analysis)

print(y_train.head())
print(X_train.head())

      completion
427          1.0
1042         1.0
115          1.0
923          1.0
1681         1.0
      Xdist  Ydist  backhand  forehand  hammer
427    -2.0   17.0         1         0       0
1042    9.0   -8.0         1         0       0
115     6.0  -15.0         0         1       0
923    -5.0   -3.0         1         0       0
1681   15.0   19.0         1         0       0


In [10]:
def train_model(X_train, X_test, y_train, y_test):
    logistic = LogisticRegressionCV(cv = 10).fit(X_train, y_train)
    
    
    print(logistic.coef_)
    
    out = logistic.predict_proba(X_test)
    return logistic

model = train_model(X_train, X_test, y_train, y_test)

C:\Users\tonyc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.0432752  -0.00434164  0.00090446 -0.00058814 -0.0002214 ]]


In [17]:
user_input = pd.DataFrame(data={ "Xdist":[30.0], "Ydist":[-5.0], "backhand":[0], "forehand":[1], "hammer":[0]})

model.predict_proba(user_input)

array([[0.11216746, 0.88783254]])